In [ ]:
!unzip "/content/drive/MyDrive/modu/archive 2.zip" -d /content/output


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/output/archive 2/indoorCVPR_09annotations/Annotations/bedroom/indoor_0430.xml  
  inflating: /content/output/__MACOSX/archive 2/indoorCVPR_09annotations/Annotations/bedroom/._indoor_0430.xml  
  inflating: /content/output/archive 2/indoorCVPR_09annotations/Annotations/bedroom/indoor_0356.xml  
  inflating: /content/output/__MACOSX/archive 2/indoorCVPR_09annotations/Annotations/bedroom/._indoor_0356.xml  
  inflating: /content/output/archive 2/indoorCVPR_09annotations/Annotations/bedroom/b22.xml  
  inflating: /content/output/__MACOSX/archive 2/indoorCVPR_09annotations/Annotations/bedroom/._b22.xml  
  inflating: /content/output/archive 2/indoorCVPR_09annotations/Annotations/bedroom/indoor_0578.xml  
  inflating: /content/output/__MACOSX/archive 2/indoorCVPR_09annotations/Annotations/bedroom/._indoor_0578.xml  
  inflating: /content/output/archive 2/indoorCVPR_09annotations/Annotations/bedroom/int712.xml  
  inflating: /content/o

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
!pip install -q -e .
!pip install -q roboflow

/content
Cloning into 'GroundingDINO'...
remote: Enumerating objects: 463, done.
remote: Counting objects: 100% (240/240), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 463 (delta 175), reused 136 (delta 136), pack-reused 223 (from 1)
Receiving objects: 100% (463/463), 12.87 MiB | 14.50 MiB/s, done.
Resolving deltas: 100% (241/241), done.
/content/GroundingDINO
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.5 MB/s eta 0:00:00


In [ ]:
import os

CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(CONFIG_PATH, "; exist:", os.path.isfile(CONFIG_PATH))

/content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py ; exist: True


In [ ]:
%cd {HOME}
!mkdir {HOME}/weights
%cd {HOME}/weights

!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

/content
/content/weights


In [ ]:
import os

WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(HOME, "weights", WEIGHTS_NAME)
print(WEIGHTS_PATH, "; exist:", os.path.isfile(WEIGHTS_PATH))

/content/weights/groundingdino_swint_ogc.pth ; exist: True


In [ ]:
%cd {HOME}/GroundingDINO

from groundingdino.util.inference import load_model, load_image, predict, annotate

model = load_model(CONFIG_PATH, WEIGHTS_PATH)

/content/GroundingDINO


final text_encoder_type: bert-base-uncased


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
!rm -rf /content/annotation


In [ ]:
import os
from groundingdino.util.inference import load_model, load_image, predict


# 모델 설정 함수
def setup_model(home_path):
    CONFIG_PATH = os.path.join(home_path, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
    WEIGHTS_PATH = "/content/weights/groundingdino_swint_ogc.pth"
    model = load_model(CONFIG_PATH, WEIGHTS_PATH)
    return model

# Bbox 텍스트 저장 함수
def bbox_to_txt(boxes, logits, phrases, class_names):
    bbox_data = []
    for idx, box in enumerate(boxes):
        class_name = phrases[idx]
        class_id = class_names.index(class_name) if class_name in class_names else -1
        if class_id != -1:
            x_center, y_center, width, height = map(float, box)
            bbox_data.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
    return bbox_data

# 이미지 처리 함수
def process_images(input_dir, output_dir, model, box_threshold, text_threshold, class_names):
    for root, _, files in os.walk(input_dir):
        for file_name in files:
            if file_name.lower().endswith(('.jpg', '.png', '.jpeg')):
                # txt 저장 경로 생성
                relative_path = os.path.relpath(root, input_dir)
                output_subdir = os.path.join(output_dir, relative_path)
                os.makedirs(output_subdir, exist_ok=True)
                txt_path = os.path.join(output_subdir, f"{os.path.splitext(file_name)[0]}.txt")

                # 이미지 경로
                image_path = os.path.join(root, file_name)
                print(f"Processing {file_name} in {root}...")

                # 이미지 로드
                image_source, image = load_image(image_path)

                # 모든 클래스에 대해 Object Detection
                bbox_output = []
                for class_name in class_names:
                    boxes, logits, phrases = predict(
                        model=model,
                        image=image,
                        caption=class_name,
                        box_threshold=box_threshold,
                        text_threshold=text_threshold
                    )

                    # bbox 텍스트 변환
                    bbox_output.extend(bbox_to_txt(boxes, logits, phrases, class_names))

                # 결과 저장
                if bbox_output:
                    with open(txt_path, "w") as txt_file:
                        txt_file.write("\n".join(bbox_output))
                    print(f"Saved bbox to {txt_path}")
                else:
                    print(f"No valid predictions for {file_name}. Skipping.")

# 메인 실행 함수
def main():
    HOME = os.getcwd()
    BASE_INPUT_DIR = "/content/output/archive 2/indoorCVPR_09/Images"
    BASE_OUTPUT_DIR = "/content/annotation"

    if not os.path.exists(BASE_OUTPUT_DIR):
        os.makedirs(BASE_OUTPUT_DIR)

    CLASS_NAMES = ['bed', 'chair', 'dresser', 'lamp', 'sofa', 'table']
    BOX_THRESHOLD = 0.35
    TEXT_THRESHOLD = 0.25

    model = setup_model(HOME)

    print(f"Processing all images in {BASE_INPUT_DIR}...")
    process_images(BASE_INPUT_DIR, BASE_OUTPUT_DIR, model, BOX_THRESHOLD, TEXT_THRESHOLD, CLASS_NAMES)

if __name__ == "__main__":
    main()


final text_encoder_type: bert-base-uncased
Processing all images in /content/output/archive 2/indoorCVPR_09/Images...
Processing AT_00_05_8295_30_l.jpg in /content/output/archive 2/indoorCVPR_09/Images/children_room...


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Processing int878.jpg in /content/output/archive 2/indoorCVPR_09/Images/livingroom...
Saved bbox to /content/annotation/livingroom/int878.txt
Processing int661.jpg in /content/output/archive 2/indoorCVPR_09/Images/livingroom...
Saved bbox to /content/annotation/livingroom/int661.txt
Processing room233.jpg in /content/output/archive 2/indoorCVPR_09/Images/livingroom...
Saved bbox to /content/annotation/livingroom/room233.txt
Processing N457044.jpg in /content/output/archive 2/indoorCVPR_09/Images/livingroom...
Saved bbox to /content/annotation/livingroom/N457044.txt
Processing int718.jpg in /content/output/archive 2/indoorCVPR_09/Images/livingroom...
Saved bbox to /content/annotation/livingroom/int718.txt
Processing roomscan35.jpg in /content/output/archive 2/indoorCVPR_09/Images/livingroom...
Saved bbox to /content/annotation/livingroom/roomscan35.txt
Processing b_salon19.jpg in /content/output/archive 2/indoorCVPR_09/Images/livingroom...
Saved bbox 

In [ ]:
# 1. 로케일 설정
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

# 2. 디렉토리 압축
!zip -r /content/annotation.zip /content/annotation

# 3. 파일 다운로드
from google.colab import files
files.download('/content/annotation.zip')

  adding: content/annotation/ (stored 0%)
  adding: content/annotation/children_room/ (stored 0%)
  adding: content/annotation/children_room/playroom_lg_35_.txt (deflated 19%)
  adding: content/annotation/children_room/OR_03_06_1002_56_l.txt (deflated 52%)
  adding: content/annotation/children_room/playroom2_30_.txt (deflated 43%)
  adding: content/annotation/children_room/OR_99_4_4922_07A_l.txt (deflated 48%)
  adding: content/annotation/children_room/AT_99_1_7838_12A_l.txt (deflated 48%)
  adding: content/annotation/children_room/fairfield_real_estate_85dogwood_playroom_750_17_.txt (deflated 43%)
  adding: content/annotation/children_room/playroom_furniture_49_.txt (deflated 43%)
  adding: content/annotation/children_room/opt_playroom_tepee_47_.txt (deflated 25%)
  adding: content/annotation/children_room/playroom7_65_.txt (deflated 51%)
  adding: content/annotation/children_room/_kids_rooms_38_.txt (deflated 46%)
  adding: content/annotation/children_room/as_playroom_40_.txt (deflat

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>